## Sentiment Analysis and Automatic Reply of 'Axis Mobile: Pay, Invest & UPI' App review using ChatGpt Prompts
#### https://www.linkedin.com/in/manali-parit-07a214251/

In [ ]:
import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

### App Id

In [ ]:
app_package = ['com.axis.mobile']

### App Detail

In [ ]:
app_info = []

for ap in tqdm(app_package):
  info = app(ap, lang='en', country='us')
  del info['comments']
  app_info.append(info)

In [ ]:
def print_json(json_object):
  json_str = json.dumps(
    json_object,
    indent=2,
    sort_keys=True,
    default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

In [ ]:
app_info_df = pd.DataFrame(app_info)
app_info_df.to_csv('AXISBANK_INFO.csv', index=None, header=True)

In [ ]:
# App Details
app_info_df.transpose()

#### Extracting the reviews

In [ ]:
app_reviews_final = []

for ap in tqdm(app_package):
  for score in list(range(1, 6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rview, _ = reviews(
        ap,
        lang='en',
        country='us',
        sort=sort_order,
        count= 1000,  # considered only 1000 reviews from each rating 
        filter_score_with=score
      )

      for r in rview:
            if sort_order == Sort.MOST_RELEVANT:
                r['sortOrder'] = 'most_relevant'
            elif sort_order == Sort.NEWEST:
                r['sortOrder'] = 'newest'
            else :
                None
        
            r['appId'] = ap

    app_reviews_final.extend(rview)

In [ ]:
# Count of reviews
len(app_reviews_final)

In [ ]:
# Converting the list into DataFrame
reviews_df = pd.DataFrame(app_reviews_final)
# Reviewing the reviews dataset we have extracted
#reviews_df.sample(5)

In [ ]:
import datetime
now = datetime.datetime.now()
current_time = now.strftime("%Y%m%d_%H%M%S")
current_time

In [ ]:
# saving the dataset in csv file
reviews_df.to_csv('AXISBANK_REVIEWS' + current_time + '.csv', index=None, header=True)

In [ ]:
reviews_df.shape

In [ ]:
reviews_df.info()

In [ ]:
reviews = reviews_df[['reviewId', 'userName' , 'content' , 'score']]
sample_df = reviews.groupby('score').apply(lambda x: x.sample(frac=0.005))
sample_df.reset_index(inplace=True,drop=True)

In [ ]:
import os
import openai
import chatgpt_cred

openai.organization = chatgpt_cred.organization
openai.api_key = chatgpt_cred.api_key
#openai.Model.list()

In [ ]:
sample_df.reset_index(inplace = True,drop = True)
sample_df['Sentiment']=None
sample_df['Anger']=None
sample_df['Reply']=None

#### chatgpt allows 3 iterations per min 

In [ ]:
n=sample_df.shape[0]

In [ ]:
import datetime
import time
import json

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


start_time = datetime.datetime.now()  # get the current time
iteration_count = 0
for i in range(n):
    text=sample_df['content'][i]
    # perform the iteration here
    iteration_count += 1
    print(f"Iteration {iteration_count}")
    
    # check if three iterations have been completed within one minute
    elapsed_time = (datetime.datetime.now() - start_time).total_seconds()
    if iteration_count % 3 == 0 and elapsed_time < 60:
        time.sleep(60 - elapsed_time)  # wait for the remaining time in the minute
        start_time = datetime.datetime.now()  # reset the start time and iteration count
        iteration_count = 0
    elif elapsed_time >= 60:
        start_time = datetime.datetime.now()  # reset the start time and iteration count
        iteration_count = 0
    
    prompt = f"""
    Identify the following items from the review text: 
    - Sentiment (positive or negative)
    - Is the customer expressing anger? (true or false)
    -Generate a reply to thank the customer for their review.
    If the sentiment is positive or neutral, thank them for \
    their text.If the sentiment is negative, apologize and suggest that \
    they can reach out to customer service.Make sure to use specific details from the text.
    Write in a concise and professional tone.

    The review is delimited with triple backticks. \
    Format your response as a json object with \
    "sentiment", "anger", 'reply' as the keys.
    Make your response as short as possible.

    Review text: '''{text}'''
    """
    response = get_completion(prompt)
    print(response)
    d=json.loads(response)
    sample_df['Sentiment'][i]=d["sentiment"]
    sample_df['Anger'][i]=d["anger"]
    sample_df['Reply'][i]=d["reply"]

In [ ]:
sample_df.to_csv('Axis_review_output.csv',index=False)

In [ ]:
sample_df